In [1]:
import os

In [2]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\unoto\\Documents\\my_project\\data_science\\Breast-Cancer-Prediction-MLOps'

In [5]:
import yaml

with open('secret.yaml', 'r') as file:
    secret = yaml.safe_load(file)

mlflow_credentials = secret.get('mlflow_credentials', {})
username = mlflow_credentials.get('username', '')
password = mlflow_credentials.get('password', '')

In [6]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Juwono136/Breast-Cancer-Prediction-MLOps.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = username
os.environ["MLFLOW_TRACKING_PASSWORD"] = password

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    scaler_path: Path
    metric_path: dict
    all_params: dict
    target_column: str
    mlflow_uri: str

In [8]:
from breastCancer.constants import *
from breastCancer.utils.common import read_yaml, create_directories, save_json

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LogisticRegression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path=config.model_path,
            scaler_path=config.scaler_path,
            metric_path=config.metric_path,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/Juwono136/Breast-Cancer-Prediction-MLOps.mlflow"
        )

        return model_evaluation_config

In [10]:
import os
import json
import pickle
import mlflow
import mlflow.sklearn
import pandas as pd
from urllib.parse import urlparse
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [11]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def log_into_mlflow(self):
        with open(self.config.model_path, 'rb') as f:
            model = pickle.load(f)
        
        with open(self.config.scaler_path, 'rb') as f:
            scaler = pickle.load(f)
        
        data = pd.read_csv(self.config.data_path)
        X = data.drop([self.config.target_column], axis=1)
        y = data[self.config.target_column]

        X = scaler.fit_transform(X)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            y_pred = model.predict(X_test)

            metrics_report = classification_report(y_test, y_pred, output_dict=True)
            save_json(path=Path(self.config.metric_path), data=metrics_report)

            mlflow.log_params(self.config.all_params)

            with open(Path(self.config.metric_path)) as f:
                metrics_report = json.load(f)
                for key, value in metrics_report.items():
                    if isinstance(value, dict):
                        for metric_name, metric_value in value.items():
                            metric_key = f"{key}_{metric_name}"
                            mlflow.log_metric(metric_key, metric_value)
                    else:
                        mlflow.log_metric(key, value)

            if tracking_url_type_store != 'file':
                mlflow.sklearn.log_model(model, "model", registered_model_name="LogisticRegressionModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [12]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-03 22:55:48,402: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-03 22:55:48,415: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-03 22:55:48,422: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-03 22:55:48,422: INFO: common: created directory at: artifacts]
[2024-03-03 22:55:48,422: INFO: common: created directory at: artifacts/model_evaluation]
[2024-03-03 22:55:50,113: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\unoto\anaconda3\envs\breast-cancer-mlops\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'LogisticRegressionModel' already exists. Creating a new version of this model...
2024/03/03 22:56:06 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: LogisticRegressionModel, version 8
Created version '8' of model 'LogisticRegressionModel'.
